<a href="https://colab.research.google.com/github/sang-ichal/Datarubrics_LLM_Evaluation/blob/main/Dataset/Translate_Math28K_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Instalasi dan Konfigurasi Awal

import json
from google.cloud import translate_v2 as translate
import os
from google.colab import files

print("Instalasi dan import selesai.")
print("Sekarang, Anda perlu mengunggah file kunci JSON akun layanan Google Cloud Anda.")
print("Pastikan Anda telah mengaktifkan 'Cloud Translation API' di proyek GCP Anda.")
print("Dan akun layanan memiliki peran 'Cloud Translation API User'.")

# Unggah file kredensial JSON akun layanan Anda
# Ini akan membuka dialog untuk Anda memilih file dari komputer Anda
uploaded = files.upload()

# Temukan nama file yang diunggah (biasanya hanya satu file)
for fn in uploaded.keys():
    CREDENTIALS_FILE = fn
    print(f"File kredensial '{CREDENTIALS_FILE}' berhasil diunggah.")
    break
else:
    print("Tidak ada file kredensial yang diunggah. Skrip mungkin gagal.")
    CREDENTIALS_FILE = None

# Set variabel lingkungan untuk kredensial
if CREDENTIALS_FILE:
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = CREDENTIALS_FILE
    print("Variabel lingkungan GOOGLE_APPLICATION_CREDENTIALS diatur.")
else:
    print("PERINGATAN: GOOGLE_APPLICATION_CREDENTIALS tidak diatur. Pastikan Anda telah mengunggah kredensial.")


# Inisialisasi klien Translate API
try:
    translate_client = translate.Client()
    print("Klien Google Cloud Translation API berhasil diinisialisasi.")
except Exception as e:
    print(f"ERROR: Gagal menginisialisasi klien Translate API. Pastikan kredensial sudah benar dan API diaktifkan. Detail: {e}")

Instalasi dan import selesai.
Sekarang, Anda perlu mengunggah file kunci JSON akun layanan Google Cloud Anda.
Pastikan Anda telah mengaktifkan 'Cloud Translation API' di proyek GCP Anda.
Dan akun layanan memiliki peran 'Cloud Translation API User'.


Saving mytranslator-464001-21e026105c72.json to mytranslator-464001-21e026105c72.json
File kredensial 'mytranslator-464001-21e026105c72.json' berhasil diunggah.
Variabel lingkungan GOOGLE_APPLICATION_CREDENTIALS diatur.
Klien Google Cloud Translation API berhasil diinisialisasi.


In [7]:
# @title Fungsi Penerjemahan Dataset

# Konfigurasi Penerjemahan
TARGET_LANGUAGE = 'id'  # Kode bahasa untuk Indonesia
SOURCE_LANGUAGE = 'zh'   # Kode bahasa untuk Mandarin (sesuai data Math23K)

def translate_text(text, target_language, source_language=None):
    """Menerjemahkan teks menggunakan Google Cloud Translation API."""
    if not text:
        return ""
    try:
        result = translate_client.translate(
            text,
            target_language=target_language,
            source_language=source_language
        )
        import html
        return html.unescape(result['translatedText'])
    except Exception as e:
        print(f"ERROR menerjemahkan teks: '{text}' - {e}")
        return text # Kembalikan teks asli jika terjadi error

def process_and_translate_dataset(input_file_name, output_file_name):
    translated_data = []

    print(f"\n--- Memulai penerjemahan untuk: {input_file_name} ---")
    try:
        with open(input_file_name, 'r', encoding='utf-8') as f:
            # --- PERUBAHAN UTAMA DI SINI ---
            # Membaca seluruh file sebagai satu array JSON
            dataset = json.load(f)
            # --- AKHIR PERUBAHAN UTAMA ---

    except FileNotFoundError:
        print(f"ERROR: File '{input_file_name}' tidak ditemukan. Pastikan sudah diunggah ke Colab.")
        return
    except json.JSONDecodeError as e:
        print(f"ERROR: Gagal memparsing JSON dari '{input_file_name}': {e}. Pastikan format file adalah array JSON valid.")
        print("Sampel file seharusnya dimulai dengan '[' dan diakhiri dengan ']'.")
        return
    except Exception as e:
        print(f"ERROR saat membaca file '{input_file_name}': {e}")
        return

    total_records = len(dataset)
    if total_records == 0:
        print(f"Tidak ada catatan valid yang ditemukan di {input_file_name}. Proses dihentikan.")
        return

    print(f"Total {total_records} catatan valid ditemukan di {input_file_name}.")

    for i, record in enumerate(dataset):
        if (i + 1) % 50 == 0 or i == 0 or i == total_records - 1: # Cetak progress setiap 50 catatan
            print(f"  Menerjemahkan catatan {i+1}/{total_records} (ID: {record.get('id', 'N/A')})...")

        # Terjemahkan 'original_text'
        original_text = record.get('original_text', '')
        if original_text:
            translated_original_text = translate_text(original_text, TARGET_LANGUAGE, SOURCE_LANGUAGE)
            record['original_text_id'] = translated_original_text
        else:
            record['original_text_id'] = ""

        # Terjemahkan 'segmented_text'
        segmented_text = record.get('segmented_text', '')
        if segmented_text:
            translated_segmented_text = translate_text(segmented_text, TARGET_LANGUAGE, SOURCE_LANGUAGE)
            record['segmented_text_id'] = translated_segmented_text
        else:
            record['segmented_text_id'] = ""

        translated_data.append(record)

    print(f"Menyimpan dataset yang diterjemahkan ke {output_file_name}...")
    with open(output_file_name, 'w', encoding='utf-8') as f:
        # Menyimpan sebagai array JSON (bukan JSON Lines lagi)
        json.dump(translated_data, f, ensure_ascii=False, indent=4) # indent=4 untuk format yang lebih rapi
    print(f"Penerjemahan untuk {input_file_name} selesai. Hasil disimpan di {output_file_name}.")

In [9]:
# @title Unggah Dataset dan Jalankan Penerjemahan

print("Silakan unggah file dataset 'math23k_train.json' dan 'math23k_test.json'.")
print("Anda dapat memilih keduanya sekaligus.")

# Unggah file dataset
uploaded_datasets = files.upload()

# Pastikan file yang dibutuhkan ada
required_files = ['math23k_train.json', 'math23k_test.json']
for f in required_files:
    if f not in uploaded_datasets:
        print(f"ERROR: File '{f}' tidak ditemukan. Mohon unggah file tersebut.")

# Proses file train
if 'math23k_train.json' in uploaded_datasets:
    process_and_translate_dataset('math23k_train.json', 'math23k_train_id.json')

# Proses file test
if 'math23k_test.json' in uploaded_datasets:
    process_and_translate_dataset('math23k_test.json', 'math23k_test_id.json')

print("\nSemua proses penerjemahan selesai. Anda dapat mengunduh file hasil dari panel file di sebelah kiri atau menggunakan tombol unduh di bawah.")

Silakan unggah file dataset 'math23k_train.json' dan 'math23k_test.json'.
Anda dapat memilih keduanya sekaligus.


Saving math23k_test.json to math23k_test.json
Saving math23k_train.json to math23k_train.json

--- Memulai penerjemahan untuk: math23k_train.json ---
Total 22162 catatan valid ditemukan di math23k_train.json.
  Menerjemahkan catatan 1/22162 (ID: 1)...
  Menerjemahkan catatan 50/22162 (ID: 51)...
  Menerjemahkan catatan 100/22162 (ID: 102)...
  Menerjemahkan catatan 150/22162 (ID: 153)...
  Menerjemahkan catatan 200/22162 (ID: 204)...
  Menerjemahkan catatan 250/22162 (ID: 258)...
  Menerjemahkan catatan 300/22162 (ID: 309)...
  Menerjemahkan catatan 350/22162 (ID: 363)...
  Menerjemahkan catatan 400/22162 (ID: 414)...
  Menerjemahkan catatan 450/22162 (ID: 466)...
  Menerjemahkan catatan 500/22162 (ID: 517)...
  Menerjemahkan catatan 550/22162 (ID: 567)...
  Menerjemahkan catatan 600/22162 (ID: 620)...
  Menerjemahkan catatan 650/22162 (ID: 671)...
  Menerjemahkan catatan 700/22162 (ID: 722)...
  Menerjemahkan catatan 750/22162 (ID: 776)...
  Menerjemahkan catatan 800/22162 (ID: 827)..